In [29]:
# imports
import os
from datetime import datetime
import shutil
from pathlib import Path
import urllib.request
import pandas as pd
from google.cloud import bigquery
from prefect import task, flow
from prefect_gcp.cloud_storage import GcsBucket

print("Setup Complete")


# Deployment 1
# Fetch the data from Github url
@task(log_prints=True, name="fetch-file-dataset_url")
def fetch(dataset_url: str):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename


# Read and tweak to fix the dtypes of pick-up and drop-off
@task(log_prints=True, name="read-and-tweak-df")
def read_tweak_df(src: str) -> pd.DataFrame:
    df = pd.read_csv(src, parse_dates=[2, 3], compression="gzip")
    return df


# Write DataFrame to a specific folder after tweaking the DataFrame
@task(log_prints=True, name="write-to-local-file")
def write_local(df: pd.DataFrame, year: int, dataset_file: str) -> Path:
    directory = Path(f"{year}")
    path_name = directory / f"{dataset_file}.parquet"
    try:
        os.makedirs(directory, exist_ok=True)
        df.to_parquet(path_name, compression="gzip", index=False)
    except OSError as error:
        print(error)
    return path_name


# https://app.prefect.cloud/account/975bd9ed-5aef-4c8a-a413-23073fef3acb/workspace/a711abba-f5ae-4b00-9315-34f92f089b77/blocks/catalog
# Upload local csv.gz file to GCS
@task(log_prints=True)
def write_gcs(path: Path) -> None:
    gcs_block = GcsBucket.load("prefect-gcs-block-ny-taxi")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    print("Loaded data to GCS...Hooray!")
    return


# Delete file after uploads
@task(log_prints=True, name="deduplicate-local-data")
def deduplicate(path: Path) -> None:
    try:
        os.remove(path)
        os.rmdir(
            path,
        )
        print("Successfully deleted directory and local files...hep hep hooray")
    except OSError as error:
        print(f"The system cannot find the file specified: {error}")
    return


@flow(log_prints=True, name="etl-web-gcs")
def etl_web_gcs(year: int, month: int):
    # Parameters
    year = 2019
    month = 5
    dataset_file = f"fhv_tripdata_{year}-{month:02}"
    dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{dataset_file}.csv.gz"

    # Execution
    # Fetch the data from Github url
    source = fetch(dataset_url)

    # no choice but to tweak the data types for us to upload this to BQ
    # Read and tweak to fix the dytpes of pick-up and drop-off
    df_tweak = read_tweak_df(source)

    # write df to local
    path_local = write_local(df_tweak, year, dataset_file)

    # Upload dataset from local to gcs
    write_gcs(path_local)

    # Delete file after uploads
    deduplicate(path_local)

    # Next step is to create a separate deployment from GCS to BigQuery


# Create Parent flow to loop
@flow(name="etl-parent-local-gcs")
def etl_parent_web_gcs(year: int, months: list):
    year = 2019
    months = [2]

    for month in months:
        etl_web_gcs(year, month)


# Run main
if __name__ == "__main__":
    etl_parent_web_gcs(year=2019, months=[2])


Setup Complete


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'fetch-file-dataset_url' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/636186100.py:17' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'read-and-tweak-df' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/636186100.py:24' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'write-to-local-file' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/636186

16:20:32.662 | INFO    | prefect.engine - Created flow run 'perky-crocodile' for flow 'etl-parent-local-gcs'

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'etl-web-gcs' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/636186100.py:67' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


16:20:35.510 | INFO    | Flow run 'perky-crocodile' - Created subflow run 'skinny-gharial' for flow 'etl-web-gcs'

...

16:21:38.497 | INFO    | Flow run 'skinny-gharial' - Executing 'deduplicate-local-data-8266ec2d-0' immediately...

16:21:39.333 | INFO    | Task run 'deduplicate-local-data-8266ec2d-0' - The system cannot find the file specified: [Errno 2] No such file or directory: '2019/fhv_tripdata_2019-05.csv.gz'

16:21:39.761 | INFO    | Task run 'deduplicate-local-data-8266ec2d-0' - Finished in state Completed()

16:21:40.128 | INFO    | Flow run 'skinny-gharial' - Finished in state Completed('All states completed.')

16:21:40.602 | INFO    | Flow run 'perky-crocodile' - Finished in state Completed('All states completed.')

In [ ]:
# import
from prefect_gcp import GcpCredentials
from google.cloud import bigquery

# Deployment 2



# Upload data from GCS to BigQuery
@flow(log_prints=True, name="etl-gcs-to-bq")
def etl_gcs_to_bq(year: int, month: int):

    gcp_creds_block = GcpCredentials.load("prefect-gcs-2023-creds")
    gcp_creds = gcp_creds_block.get_credentials_from_service_account()
    client = bigquery.Client(credentials=gcp_creds)
    table_id = "dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        schema=[
            bigquery.SchemaField("dispatching_base_num", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("pickup_datetime", "TIMESTAMP", mode="NULLABLE"),
            bigquery.SchemaField("dropOff_datetime", "TIMESTAMP", mode="NULLABLE"),
            bigquery.SchemaField("PUlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("DOlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField(
                "SR_Flag",
                "FLOAT",
                mode="NULLABLE",
            ),
            bigquery.SchemaField("Affiliated_base_number", "STRING", mode="NULLABLE"),
        ],
    )
    uri = f"gs://ny_taxi_bucket_de_2023/{year}/fhv_tripdata_{year}-{month:02}.csv.gz"

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows.")


# Parent flow ETL
@flow(log_prints=True, name="etl-parent-to-bq")
def etl_parent_bq_flow(
    year: int = 2019, months: list[int] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
):
    for month in months:
        etl_gcs_to_bq(year, month)


# run main
if __name__ == "__main__":
    year = 2019
    months = [3]

    etl_parent_bq_flow(year, months)

In [14]:
src = "/Users/reneboygarcia/Downloads/fhv_tripdata_2019-02.csv"
df = pd.read_csv(src)
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00037,2019-02-01 00:08:44,2019-02-01 00:23:35,264.0,265.0,NaN,B00037
1,B00037,2019-02-01 00:27:51,2019-02-01 00:32:54,264.0,265.0,NaN,B00037
2,B00037,2019-02-01 00:18:30,2019-02-01 00:25:45,264.0,265.0,NaN,B00037
3,B00037,2019-02-01 00:43:15,2019-02-01 00:48:29,264.0,265.0,NaN,B00037
4,B00037,2019-02-01 00:01:45,2019-02-01 00:09:13,264.0,265.0,NaN,B00037


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1707649 entries, 0 to 1707648
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   dispatching_base_num    object 
 1   pickup_datetime         object 
 2   dropOff_datetime        object 
 3   PUlocationID            float64
 4   DOlocationID            float64
 5   SR_Flag                 float64
 6   Affiliated_base_number  object 
dtypes: float64(3), object(4)
memory usage: 91.2+ MB
